In [ ]:
import torch
from torchid.dynonet.module.lti import SisoLinearDynamicalOperator
from torchid.dynonet.module.static import SisoStaticNonLinearity
import nonlinear_benchmarks
from nonlinear_benchmarks import error_metrics as metrics
import matplotlib.pyplot as plt
import control

In [ ]:
# Load training results
ckpt = torch.load("ckpt.pt")
cfg = ckpt["cfg"]

In [ ]:
# Instantiate model and load trained parameters
model = torch.nn.Sequential(
    SisoLinearDynamicalOperator(cfg.n_b, cfg.n_a, n_k=1),
    SisoStaticNonLinearity(n_hidden=10, activation="tanh"),
    SisoLinearDynamicalOperator(cfg.n_b, cfg.n_a),
)
model.load_state_dict(ckpt["model"])

In [ ]:
# Load test data
_, test = nonlinear_benchmarks.WienerHammerBenchMark()
u_test, y_test = test
u_test = u_test.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
ts = test.sampling_time

In [ ]:
# Load scalers
scaler_u = ckpt["scaler_u"]
scaler_y = ckpt["scaler_y"]

In [ ]:
# Scale u, simulate, and inverse scale result
ut = torch.tensor(scaler_u.transform(u_test)).unsqueeze(0).float()
with torch.no_grad():
    y_test_hat = model(ut).squeeze(0).numpy()

y_test_hat = scaler_y.inverse_transform(y_test_hat)

In [ ]:
# Plot results
plt.figure()
plt.plot(y_test, "k")
plt.plot(y_test_hat, "b")
plt.plot(y_test_hat - y_test, "r")
plt.show()

In [ ]:
#mse = np.mean((y_test_hat - y_test)**2)
#rmse = np.sqrt(mse) * 1000
rmse = metrics.RMSE(y_test_hat, y_test)[0]*1000
fit = metrics.fit_index(y_test_hat, y_test)[0]
print(f"{rmse=:.2f} mV\n{fit=:.1f} %") 

In [ ]:
# Inspect blocks
G1 = model[0]
G1_num, G1_den = G1.get_tfdata()
G1_sys = control.TransferFunction(G1_num, G1_den, ts)

plt.figure()
mag_G1, phase_G1, omega_G1 = control.bode(G1_sys, omega_limits=[1e2, 1e5])
plt.suptitle("$G_1$ bode plot");

In [ ]:
G2 = model[2]
G2_num, G2_den = G1.get_tfdata()
G2_sys = control.TransferFunction(G2_num, G2_den, ts)

plt.figure()
mag_G2, phase_G2, omega_G2 = control.bode(G1_sys, omega_limits=[1e2, 1e5])
plt.suptitle("$G_1$ bode plot");

In [ ]:
F = model[1]
with torch.no_grad():
    in_nl = G1(ut)
    out_nl = F(in_nl)

in_nl = in_nl.squeeze(0)
out_nl = out_nl.squeeze(0)

plt.figure()
plt.plot(in_nl, out_nl, 'b*')
plt.xlabel('Static non-linearity input (-)')
plt.ylabel('Static non-linearity input (-)')
plt.grid(True)